In [1]:
import boto3
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

In [ ]:
# TODO
# Artist seed needs to search song first and get value that best matches target value 
# Normalize values
# TODO handle case where recommendation doesn't return anything

In [104]:
client = boto3.client('lambda')
response = client.invoke(FunctionName='spotifyLambda', InvocationType='RequestResponse')
data = json.loads(response['Payload'].read())
token = json.loads(data['body'])['token']

In [105]:
sp = spotipy.Spotify(token)

In [4]:
def getFeatures(songID):
    audioFeaturesRaw = sp.audio_features(songID)[0]    
    return audioFeaturesRaw

In [100]:
def getRecommendations(ids, **kwargs):
#     print("Recommendations: ", ids, kwargs)
    recommendations = sp.recommendations(seed_tracks=ids, **kwargs)
    
    return recommendations['tracks']

In [118]:
danceability = [.60, .70, .92, .73, .65]
energy = [.60, .70, .92, .73, .65]
targets: {
        'target_danceability': danceability,
        'target_energy': energy
}
seeds = {
    'location': 1,
    'id': 'spotify:track:22bNlvnDQbYAq0Ex2sduSu'

}
playlist = generatePlaylist(5, targets, seeds)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [111]:
def generatePlaylist(numTracks, targets, seed):
    import copy
    actual = copy.deepcopy(targets)
    
    """
    numTracks: number of tracks in the set
    targets: dictionary of target values
    seeds: dictionary of spotify id of artist or song, and location
    """
    playlist = np.empty(shape=numTracks, dtype=np.object)
    
    seedIndex = seed['location']
    features = getFeatures(seed['id'])
    playlist[seedIndex] = seed['id']
    for metric in targets:
        targets[metric] = np.asarray(targets[metric], dtype=np.double)
        features = getFeatures(seed['id']) 
        offset = features[metric.split('_')[1]] - targets[metric][seedIndex]
        if offset > 0:
            targets[metric] = targets[metric]+offset
        if offset < 0:
            targets[metric] = targets[metric]-offset
    
            
    for i in range(seedIndex+1, len(playlist)):
        
        k = {}
        for metric in targets:
            k[metric] = targets[metric][i]        
        num_seeds = min(i-seedIndex, 5)
        trackseeds = playlist[seedIndex:seedIndex+num_seeds]
        recommendations = getRecommendations(list(trackseeds), **k)
        for track in recommendations:
            feature_val_diff = -1
            if track['id'] not in playlist:
                playlist[i] = track['id']
                break

    for i in range(seedIndex-1, -1, -1):
        
        k = {}
        for metric in targets:
            k[metric] = targets[metric][i]        
        x = len(playlist)-i
        if x >= 5:
            x = 5
        trackseeds = playlist[i+1:i+x+1]
        recommendations = getRecommendations(list(trackseeds), **k)
        for track in recommendations:
            if track['id'] not in playlist:
                playlist[i] = track['id']
                break
    
#     actual = []
#     for i in range(0, len(playlist)):
#         features = getFeatures(playlist[i])
#         actual.append(features['danceability'])
#     print(targets['target_danceability'])
#     print(actual)
    
#     actual = []
#     for i in range(0, len(playlist)):
#         features = getFeatures(playlist[i])
#         actual.append(features['energy'])
#     print(targets['target_energy'])
#     print(actual)
    resultAnalysis(playlist, targets)
    return playlist  

In [130]:
def resultAnalysis(playlist, targets):
    
    features = []
    
    for i in range(0, len(playlist)):
        features.append(getFeatures(playlist[i]))
        
    for metric in targets:
        actual = []
        for j in range(0, len(targets[metric])):
            actual.append(features[j][metric.split('_')[1]])
        print(f"TARGET {metric.split('_')[1]}: {targets[metric]}")
        print(f"ACTUAL {metric.split('_')[1]}: {actual}")
        
        

In [133]:
danceability = [.60, .70, .92, .73, .65]
energy = [.60, .65, .82, .65, .50]
valence = [.60, .65, .75, .65, .65]
targets= {
        'target_danceability': danceability,
        'target_energy': energy,
        'target_valence': valence
}
seeds = {
    'location': 1,
    'id': 'spotify:track:22bNlvnDQbYAq0Ex2sduSu'

}
playlist = generatePlaylist(5, targets, seeds)

TARGET danceability: [0.676 0.776 0.996 0.806 0.726]
ACTUAL danceability: [0.698, 0.776, 0.8, 0.808, 0.718]
TARGET energy: [0.803 0.853 1.023 0.853 0.703]
ACTUAL energy: [0.706, 0.853, 0.89, 0.853, 0.693]
TARGET valence: [1.2175 1.2675 1.3675 1.2675 1.2675]
ACTUAL valence: [0.745, 0.0325, 0.683, 0.677, 0.664]


In [38]:
sp.user_playlist_create(name="Test Playlist", user="jup118")

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0qxanOC7s6PZeai31xM2hi'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0qxanOC7s6PZeai31xM2hi',
 'id': '0qxanOC7s6PZeai31xM2hi',
 'images': [],
 'name': 'Test Playlist',
 'owner': {'display_name': 'jup118',
  'external_urls': {'spotify': 'https://open.spotify.com/user/jup118'},
  'href': 'https://api.spotify.com/v1/users/jup118',
  'id': 'jup118',
  'type': 'user',
  'uri': 'spotify:user:jup118'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw0YjM4NzA1ZGNhNThjMjk2Njg2MGQxNGE2MGU3ZDUwYmY0N2U1OGJh',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/0qxanOC7s6PZeai31xM2hi/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:0qxanOC7s6PZeai31xM2hi'}

In [134]:
sp.playlist_add_items(items=playlist, playlist_id="0qxanOC7s6PZeai31xM2hi")

{'snapshot_id': 'OSwwOWNiZWI1MzkwZmEyNjIyOTI2OWM5ZmQxNmJlMzQzNjhhYTkyODUy'}

In [56]:
playlist

array(['3ioD2gvfwj4YoOKhxS1EgN', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu',
       '1LxenWKrYzbX8zvbuZgKHn', '6IgIPWjUrRBTIm07t7H64S',
       '5esTnUp3cLgKsU2Etci2Ao'], dtype=object)